In [4]:
"""
File v422_binary_SM_base.py

Michel Bierlaire
Tue Aug  4 20:09:59 2020
"""

# Import the packages
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
from biogeme.expressions import Beta

# Read the data
df = pd.read_csv('swissmetro.dat', '\t')
database = db.Database('swissmetro', df)

# The following statement allows you to use the names of the
# variable as Python variable.
globals().update(database.variables)

# Define new variables
CAR_AV_SP = CAR_AV * (SP != 0)
TRAIN_AV_SP = TRAIN_AV * (SP != 0)

# If the person has a GA (season ticket) her incremental cost is actually 0
# rather than the cost value gathered from the
# network data.

TRAIN_COST = TRAIN_CO * (GA == 0)

# For numerical reasons, it is good practice to scale the data to
# that the values of the parameters are around 1.0.
# A previous estimation with the unscaled data has generated
# parameters around -0.01 for both cost and time. Therefore, time and
# cost are multipled my 0.01.
TRAIN_TT_SCALED = TRAIN_TT / 100
TRAIN_COST_SCALED = TRAIN_COST / 100
CAR_TT_SCALED = CAR_TT / 100
CAR_CO_SCALED = CAR_CO / 100

SENIOR = AGE == 5

# Removing some observations
# All observations verifying the following expression will not be
# considered for estimation
# The modeler here has developed the model only for trips to work.
# Observations such that the dependent variable CHOICE is 0 and
# CHOICE is 2 are also removed.
exclude = (TRAIN_AV_SP == 0) + \
    (CAR_AV_SP == 0) + \
    (CHOICE == 0) + \
    (CHOICE == 2) + \
    ((PURPOSE != 1) * (PURPOSE != 3)) > 0
database.remove(exclude)


# Parameters to be estimated
# Arguments:
#   - 1  Name for report; Typically, the same as the variable.
#   - 2  Starting value.
#   - 3  Lower bound.
#   - 4  Upper bound.
#   - 5  0: estimate the parameter, 1: keep it fixed.

ASC_CAR = Beta('ASC_CAR', 0, None, None, 0)
ASC_TRAIN = Beta('ASC_TRAIN', 0, None, None, 1)
B_TIME = Beta('B_TIME', 0, None, None, 0)
B_CAR_COST = Beta('B_CAR_COST', 0, None, None, 0)
B_TRAIN_COST = Beta('B_TRAIN_COST', 0, None, None, 0)

B_HE = Beta('B_HE', 0, None, None, 0)

B_SENIOR = Beta('B_SENIOR', 0, None, None, 0)
B_GA = Beta('B_GA', 0, None, None, 0)


# Utility functions
V1 = ASC_TRAIN + B_TIME * TRAIN_TT_SCALED + B_TRAIN_COST * TRAIN_COST_SCALED + B_HE * TRAIN_HE + B_SENIOR * SENIOR + B_GA * GA
V3 = ASC_CAR + B_TIME * CAR_TT_SCALED + B_CAR_COST * CAR_CO_SCALED

# Associate utility functions with the numbering of alternatives
V = {1: V1,
     3: V3}

# Associate the availability conditions with the alternatives

av = {1: TRAIN_AV_SP,
      3: CAR_AV_SP}

# The choice model is a logit, with availability conditions
logprob = models.loglogit(V, av, CHOICE)

# Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'binary_SM_base'

# Estimate the parameters
results = biogeme.estimate()

print(results.getEstimatedParameters())

                 Value   Std err    t-test       p-value  Rob. Std err  \
ASC_CAR      -0.016370  0.198630 -0.082414  9.343171e-01      0.261616   
B_CAR_COST   -0.283286  0.176779 -1.602481  1.090492e-01      0.220558   
B_GA          1.739498  0.242354  7.177516  7.098766e-13      0.288887   
B_HE         -0.006822  0.001826 -3.735763  1.871468e-04      0.001821   
B_SENIOR      1.539771  0.203085  7.581883  3.397282e-14      0.194706   
B_TIME       -0.754232  0.148846 -5.067211  4.036866e-07      0.445395   
B_TRAIN_COST -1.552998  0.172887 -8.982752  0.000000e+00      0.287366   

              Rob. t-test  Rob. p-value  
ASC_CAR         -0.062573  9.501069e-01  
B_CAR_COST      -1.284406  1.990000e-01  
B_GA             6.021378  1.729384e-09  
B_HE            -3.745527  1.800155e-04  
B_SENIOR         7.908184  2.664535e-15  
B_TIME          -1.693398  9.037966e-02  
B_TRAIN_COST    -5.404253  6.507895e-08  


In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('swissmetro.dat', '\t')
df

,GROUP,SURVEY,SP,ID,PURPOSE,FIRST,TICKET,WHO,LUGGAGE,AGE,...,TRAIN_TT,TRAIN_CO,TRAIN_HE,SM_TT,SM_CO,SM_HE,SM_SEATS,CAR_TT,CAR_CO,CHOICE
0,2,0,1,1,1,0,1,1,0,3,...,112,48,120,63,52,20,0,117,65,2
1,2,0,1,1,1,0,1,1,0,3,...,103,48,30,60,49,10,0,117,84,2
2,2,0,1,1,1,0,1,1,0,3,...,130,48,60,67,58,30,0,117,52,2
3,2,0,1,1,1,0,1,1,0,3,...,103,40,30,63,52,20,0,72,52,2
4,2,0,1,1,1,0,1,1,0,3,...,130,36,60,63,42,20,0,90,84,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10723,3,1,1,1192,4,1,7,1,0,5,...,148,13,30,93,17,30,0,156,56,2
10724,3,1,1,1192,4,1,7,1,0,5,...,148,12,30,96,16,10,0,96,70,3
10725,3,1,1,1192,4,1,7,1,0,5,...,148,16,60,93,16,20,0,96,56,3
10726,3,1,1,1192,4,1,7,1,0,5,...,178,16,30,96,17,30,0,96,91,2
